In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader, PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from kiwipiepy import Kiwi

# reorder
from langchain_community.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda

# multi-query
from langchain.retrievers.multi_query import MultiQueryRetriever

# reranker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

kiwi = Kiwi()

In [3]:
#from langchain.retrievers.document_compressors import Cross

In [4]:
# kiwi 함수
def kiwi_tokenize(docs):
    return [token.form for token in kiwi.tokenize(docs)]

In [5]:
loaders = [
    PDFPlumberLoader("data/RAG/Adaptive_RAG.pdf"),
    PDFPlumberLoader("data/RAG/Naive_RAG.pdf"),
    PDFPlumberLoader('data/RAG/RAPTOR_RAG.pdf'),
    PDFPlumberLoader('data/RAG/Self_RAG.pdf')
]

docs = []

for loader in loaders:
    docs.extend(loader.load())

In [6]:
len(docs)

87

In [7]:
# Ensemble Retriever (kiwi bm25 + chroma similarity)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

documents = text_splitter.split_documents(docs)

chroma_vector_store = Chroma.from_documents(documents, embedding)
chroma_retriever = chroma_vector_store.as_retriever()

kiwi_vector_store = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, kiwi_vector_store],
    weights=[0.5, 0.5],
    search_kwargs={"k": 10}
    #search_type="mmr"
)

In [8]:
# MultiQUery Retriever

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = ensemble_retriever,
    llm=llm
)

In [9]:
# Reranker

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개 모델 선택
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor, base_retriever = ensemble_retriever #multiquery_retriever
)

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
# Reorder
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [11]:
# prompt
template = """
    당신은 주어진 문서에 대해서 QA를 해주는 assistant bot입니다.
    주어진 문서를 이용해서 답변해 주세요
    만약 주어진 질문에 대해서 모른다면, 모른다고 답변해 주세요
    문서의 출처와 페이지 넘버를 작성해주세요
    한국어로 답변해주세요

    #Example Format:
    (brief summary of the answer)
    (table)
    (detailed answer to the question)

    **출처**
    - (page source and page number)

    # Question:
    {question}

    # Context:
    {context}

    # Answer
"""
prompt = PromptTemplate.from_template(template, input_variable=["question", "context"])

In [12]:
# chain
chain = ({"context":compression_retriever| RunnableLambda(reorder_documents), "question": RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [13]:
print(chain.invoke("AdaptiveRAG에 대해서 설명해 주세요"))

Adaptive-RAG는 반복적으로 검색기(retriever)와 대형 언어 모델(LLM)에 접근하여 Chain-of-Thought 추론을 수행하는 방식으로, 문제의 해결책을 도출하거나 최대 단계 수에 도달할 때까지 진행됩니다. 이 모델은 복잡한 쿼리를 완벽하게 분류할 수 있는 오라클 분류기를 갖춘 이상적인 시나리오에서 작동합니다. Adaptive-RAG는 적응형 전략의 필요성을 강조하며, 최적의 성능을 달성하기 위해 개선된 분류기를 개발하는 방향을 제안합니다.

| Adaptive-RAG의 특징 | 설명 |
|---------------------|------|
| 접근 방식          | 검색기와 LLM의 반복적 접근 |
| 추론 방식          | Chain-of-Thought 추론 |
| 성능               | 효율적이고 효과적임 |
| 구현 세부사항      | A100 GPU 사용, PyTorch 및 Transformers 라이브러리로 구현 |

Adaptive-RAG는 복잡한 쿼리 처리에서 뛰어난 성능을 보이며, 다양한 검색 증강 생성 접근 방식과 비교했을 때도 우수한 성과를 나타냅니다.

**출처**
- data/RAG/Adaptive_RAG.pdf, 페이지 6, 13


In [14]:
print(chain.invoke("Self-RAG에 대해서 설명해 주세요"))

SELF-RAG는 대형 언어 모델(LLM)의 품질과 사실성을 향상시키기 위해 설계된 새로운 프레임워크입니다. 이 방법은 필요에 따라 정보를 검색하고, 생성하며, 자기 반성을 통해 텍스트 구문을 비판하는 과정을 포함합니다. SELF-RAG는 LLM이 자신의 생성 과정을 반영하도록 학습하여, 더 높은 품질의 응답을 생성할 수 있도록 돕습니다.

| SELF-RAG의 주요 특징 |
|---------------------|
| 1. 필요에 따른 정보 검색 |
| 2. 자기 반성을 통한 비판적 생성 |
| 3. 사실성 향상 |
| 4. LLM의 유연성 유지 |

SELF-RAG는 LLM이 주어진 작업 입력에 대해 출력과 중간의 특별한 출력을 생성함으로써 자신의 생성 과정을 반영하도록 훈련됩니다. 이 과정은 LLM이 더 높은 품질의 응답을 생성하고, 사실적 정확성을 높이며, 불필요한 정보 검색을 줄이는 데 기여합니다. 연구 결과, SELF-RAG는 기존의 RAG 방법보다 더 나은 성능을 보였습니다.

**출처**
- Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection, 페이지 9.
